<a href="https://colab.research.google.com/github/FuzzilyDeveloper/ML_Project/blob/master/obesity_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane
!pip install keras-metrics
!pip install silence-tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 65.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for silence-tensorflow: filename=silence_tensorflow-1.2.1-py3-none-any.whl size=4465 sha256=f91f4b15dd66af0417520f10612a09f4c48d6ff0983ba2eb5a6d27d475b072af
  Stored in directory: /root/.cache/pip/wheels/7d/2c/24/e130d6102c0df56631b9db7479d9a6a53c5d97fb06b5f61b98
  Created wheel for support_developer: filename=support_developer-1.0.5-py3-none-any.whl size=5630 sha256=a0b507bdc8e969f7c0f188e2393f34b7aad926822ee1f663f89fb4a203edec9d
  Stored in directory: /root/.cache/pip/wheels/b6/72/c8/3054a5897ba0713dfa7a941364d68cbd42b0755c8e2ec1c18c
Successfully built silence-tensorflow sup

In [ ]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf


import keras_metrics
import matplotlib.pyplot as plt


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

ob_train = pd.read_csv('/content/drive/MyDrive/playground-series-s4e2/train.csv')
ob_train

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II
20756,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile,Overweight_Level_II


In [ ]:
ob_test = pd.read_csv('/content/drive/MyDrive/playground-series-s4e2/test.csv')
ob_test

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,34593,Male,23.327836,1.721384,78.030383,yes,no,2.813234,3.000000,Sometimes,no,1.000000,no,0.807076,0.778632,Sometimes,Public_Transportation
13836,34594,Female,29.000000,1.590000,62.000000,no,yes,3.000000,3.000000,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation
13837,34595,Female,22.935612,1.585547,44.376637,no,yes,3.000000,2.273740,Frequently,no,2.000000,no,1.949840,1.000000,Sometimes,Public_Transportation
13838,34596,Male,21.000000,1.620000,53.000000,yes,yes,2.000000,3.000000,Sometimes,no,2.000000,no,3.000000,2.000000,no,Public_Transportation


In [ ]:
ob_test[0:35]

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation
5,20763,Male,19.799054,1.844751,59.605028,yes,yes,2.000000,4.000000,Sometimes,no,2.722063,no,2.000000,1.283673,no,Automobile
6,20764,Male,18.000000,1.728834,51.442293,yes,yes,1.202075,3.000000,Sometimes,no,1.087166,no,0.788585,1.000000,Sometimes,Public_Transportation
7,20765,Male,20.000000,1.810000,80.000000,yes,yes,2.000000,3.000000,Always,no,2.000000,no,3.000000,0.000000,no,Automobile
8,20766,Male,21.000000,1.700000,80.000000,yes,yes,2.000000,3.000000,Frequently,yes,2.000000,no,0.000000,2.000000,no,Public_Transportation
9,20767,Female,21.000000,1.560000,53.000000,no,yes,2.000000,3.000000,Sometimes,no,2.000000,no,0.000000,1.000000,Sometimes,Public_Transportation


In [ ]:
# prompt: apply label encoder sklearn

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

ob_train['NObeyesdad'] = le.fit_transform(ob_train['NObeyesdad'])
#ob_test['target'] = le.transform(ob_test['target'])


In [ ]:
ob_train['NObeyesdad'].unique()

array([6, 1, 0, 4, 3, 5, 2])

In [ ]:
X = ob_train.drop(['id','NObeyesdad'], axis=1)
y = ob_train['NObeyesdad']

In [ ]:
y.unique()

array([6, 1, 0, 4, 3, 5, 2])

In [ ]:
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

In [ ]:
numerical_cols

['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

In [ ]:
# One-hot encode categorical variables
train_data = pd.get_dummies(X, columns=['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS'])
test_data = pd.get_dummies(ob_test, columns=['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS'])

In [ ]:
test_data

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,...,SCC_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,20758,26.899886,1.848294,120.644178,2.938616,3.000000,2.825629,0.855400,0.000000,0,...,0,0,0,1,0,0,0,0,1,0
1,20759,21.000000,1.600000,66.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1,...,0,0,0,1,0,0,0,0,1,0
2,20760,26.000000,1.643355,111.600553,3.000000,3.000000,2.621877,0.000000,0.250502,1,...,0,0,0,1,0,0,0,0,1,0
3,20761,20.979254,1.553127,103.669116,2.000000,2.977909,2.786417,0.094851,0.000000,0,...,0,0,0,1,0,0,0,0,1,0
4,20762,26.000000,1.627396,104.835346,3.000000,3.000000,2.653531,0.000000,0.741069,1,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,34593,23.327836,1.721384,78.030383,2.813234,3.000000,1.000000,0.807076,0.778632,0,...,0,0,0,1,0,0,0,0,1,0
13836,34594,29.000000,1.590000,62.000000,3.000000,3.000000,2.000000,0.000000,0.000000,1,...,0,0,0,1,0,0,0,0,1,0
13837,34595,22.935612,1.585547,44.376637,3.000000,2.273740,2.000000,1.949840,1.000000,1,...,0,0,0,1,0,0,0,0,1,0
13838,34596,21.000000,1.620000,53.000000,2.000000,3.000000,2.000000,3.000000,2.000000,0,...,0,0,0,0,1,0,0,0,1,0


In [ ]:
from sklearn.preprocessing import StandardScaler
#X_data_norm = StandardScaler().fit_transform(X_data)
train_data[numerical_cols]= StandardScaler().fit_transform(train_data[numerical_cols])

In [ ]:
train_data

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,...,SCC_no,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,0.105699,-0.002828,-0.235713,-0.836279,0.314684,1.206594,-1.171141,0.597438,0,1,...,1,0,0,1,0,0,0,0,1,0
1,-1.027052,-1.606291,-1.170931,-0.836279,0.338364,-0.048349,0.021775,0.636513,1,0,...,1,0,0,0,1,1,0,0,0,0
2,-1.027052,0.128451,-1.430012,-1.060332,-1.913423,-0.195644,-0.138022,1.755239,1,0,...,1,0,0,0,1,0,0,0,1,0
3,-0.507929,0.120090,1.644770,1.039171,0.338364,-0.584035,0.579896,0.271455,1,0,...,1,0,0,1,0,0,0,0,1,0
4,1.371197,2.450367,0.224054,0.438397,-1.119801,-0.081469,1.176486,0.523111,0,1,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,0.227725,0.760293,0.996987,0.888355,0.338364,0.201151,0.416056,-0.697686,0,1,...,1,0,0,1,0,0,0,0,1,0
20754,-1.027052,0.111729,-1.436296,1.039171,1.756085,-1.691863,1.214691,0.636513,0,1,...,1,0,0,1,0,0,0,0,1,0
20755,-0.657669,1.366537,0.670717,-0.071439,0.338364,-0.048349,0.210303,0.966092,0,1,...,1,0,0,0,1,0,0,0,1,0
20756,1.760067,-0.002805,-0.165574,0.422594,-1.119801,0.189694,-1.171141,0.593055,0,1,...,1,0,0,0,1,1,0,0,0,0


In [ ]:
train_data.columns

Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE',
       'Gender_Female', 'Gender_Male', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes',
       'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no',
       'SMOKE_no', 'SMOKE_yes', 'SCC_no', 'SCC_yes', 'CALC_Frequently',
       'CALC_Sometimes', 'CALC_no', 'MTRANS_Automobile', 'MTRANS_Bike',
       'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking'],
      dtype='object')

In [ ]:
test_ids = test_data["id"]


In [ ]:
len(test_ids)

13840

In [ ]:
test_data = test_data.drop(['id','CALC_Always'], axis=1)

In [ ]:
test_data[numerical_cols]= StandardScaler().fit_transform(test_data[numerical_cols])

In [ ]:
test_data.columns

Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE',
       'Gender_Female', 'Gender_Male', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes',
       'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no',
       'SMOKE_no', 'SMOKE_yes', 'SCC_no', 'SCC_yes', 'CALC_Frequently',
       'CALC_Sometimes', 'CALC_no', 'MTRANS_Automobile', 'MTRANS_Bike',
       'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking'],
      dtype='object')

In [ ]:
test_data

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,...,SCC_no,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,0.508163,1.682789,1.273786,0.932524,0.350808,1.298389,-0.141769,-1.005016,0,1,...,1,0,0,1,0,0,0,0,1,0
1,-0.509128,-1.114658,-0.818988,-0.833161,-2.462523,1.583678,0.030307,-1.005016,1,0,...,1,0,0,1,0,0,0,0,1,0
2,0.353000,-0.626191,0.927432,1.047997,0.350808,0.965029,-1.159701,-0.592995,1,0,...,1,0,0,1,0,0,0,0,1,0
3,-0.512705,-1.642760,0.623672,-0.833161,0.319733,1.234234,-1.046828,-1.005016,0,1,...,1,0,0,1,0,0,0,0,1,0
4,0.353000,-0.805996,0.668336,1.047997,0.350808,1.016818,-1.159701,0.213881,1,0,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,-0.107749,0.252936,-0.358246,0.696661,0.350808,-1.688533,-0.199274,0.275664,0,1,...,1,0,0,1,0,0,0,0,1,0
13836,0.870277,-1.227324,-0.972181,1.047997,0.350808,-0.052428,-1.159701,-1.005016,1,0,...,1,0,0,1,0,0,0,0,1,0
13837,-0.175379,-1.277495,-1.647125,1.047997,-0.670797,-0.052428,1.160623,0.639766,1,0,...,1,0,0,1,0,0,0,0,1,0
13838,-0.509128,-0.889324,-1.316865,-0.833161,0.350808,-0.052428,2.410322,2.284548,0,1,...,1,0,0,0,1,0,0,0,1,0


In [ ]:
test_data.shape

(13840, 30)

In [ ]:
y

0        6
1        1
2        0
3        4
4        6
        ..
20753    3
20754    0
20755    3
20756    6
20757    3
Name: NObeyesdad, Length: 20758, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_data, y, test_size=0.2, random_state=42)



In [ ]:
X_train

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,...,SCC_no,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
9958,-1.202863,0.798937,0.345438,1.039171,0.338364,-0.048349,2.407607,0.636513,0,1,...,1,0,0,0,1,1,0,0,0,0
7841,-0.206438,0.608684,-1.251408,-0.836279,1.756085,-0.048349,1.214691,0.636513,0,1,...,1,0,0,0,1,0,0,0,1,0
9293,-0.352234,1.370087,1.294114,1.039171,0.169396,-0.631241,-0.288525,-1.024344,0,1,...,1,0,0,1,0,0,0,0,1,0
15209,3.016595,-1.377221,-0.299019,-0.836279,0.338364,-1.691863,-1.171141,-1.024344,1,0,...,1,0,0,1,0,1,0,0,0,0
16515,-0.147998,1.142541,0.269619,1.039171,0.338364,1.595165,1.214691,0.636513,0,1,...,1,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,-1.027052,0.254451,-0.282234,0.342987,-0.281329,-0.303677,-0.827543,0.175253,1,0,...,1,0,0,0,1,0,0,0,1,0
11964,-0.323809,0.569868,0.269619,1.039171,0.338364,-0.048349,2.407607,-1.024344,0,1,...,1,0,0,1,0,0,0,0,1,0
5390,-0.499620,-0.919083,-0.753930,-0.836279,0.338364,1.595165,1.214691,-1.024344,0,1,...,1,0,0,1,0,0,0,0,1,0
860,0.379434,-0.574047,0.911789,1.039171,0.338364,1.218360,-1.171141,-0.630211,1,0,...,1,0,0,1,0,0,0,0,1,0


In [ ]:
X_train.shape[1]

30

In [ ]:
# from keras import regularizers
# #from keras.callbacks import ModelCheckpoint

# n_qubits = 1
# layers = 3
# data_dimension = 1
# dev = qml.device("default.qubit",wires = n_qubits)

# @qml.qnode(dev,interface = "tf")

# def qnode(inputs, weights):
#   qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
#   qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
#   return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# weight_shapes = { "weights" : (layers, n_qubits,3)}

# l2 = tf.keras.regularizers.L2(0.02)
# #layer = Dense(units=4, kernel_regularizer=regularizer)

# qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim = n_qubits)
# flatlayer = tf.keras.layers.Flatten()
# clayer1 = tf.keras.layers.Dense(64,input_shape=(X_train.shape[1],))
# clayer2 = tf.keras.layers.Dense(64)
# clayer3 = tf.keras.layers.Dense(16,activation = "relu")
# drop1 = tf.keras.layers.Dropout(0.05)
# clayer4 = tf.keras.layers.Dense(1)
# #model = tf.keras.models.Sequential([clayer1,qlayer,clayer2])

# #---------------------------------------
# #model = tf.keras.models.Sequential()
# #model.add(tf.keras.layers.Dense(8, input_dim=1, activation='relu'))
# #model.add(tf.keras.layers.Dense(7, activation='softmax'))
#  # Compile model
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#  #--------------------------------------------------------
# model = tf.keras.models.Sequential([clayer1,clayer2,clayer4])



# opt = tf.keras.optimizers.Adam(learning_rate=0.001)

# model.compile(opt,loss="categorical-crossentropy",metrics=["loss"])

In [ ]:
# prompt: create a very basic neural network for multiclass classification

model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, input_shape=(X_train.shape[1],), activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)


Epoch 1/10
519/519 [==============================] - 2s 3ms/step - loss: 0.9983 - accuracy: 0.6270
Epoch 2/10
519/519 [==============================] - 1s 2ms/step - loss: 0.5100 - accuracy: 0.8154
Epoch 3/10
519/519 [==============================] - 1s 2ms/step - loss: 0.4246 - accuracy: 0.8515
Epoch 4/10
519/519 [==============================] - 1s 2ms/step - loss: 0.3973 - accuracy: 0.8620
Epoch 5/10
519/519 [==============================] - 1s 2ms/step - loss: 0.3864 - accuracy: 0.8688
Epoch 6/10
519/519 [==============================] - 1s 2ms/step - loss: 0.3810 - accuracy: 0.8685
Epoch 7/10
519/519 [==============================] - 1s 2ms/step - loss: 0.3757 - accuracy: 0.8703
Epoch 8/10
519/519 [==============================] - 1s 2ms/step - loss: 0.3721 - accuracy: 0.8717
Epoch 9/10
519/519 [==============================] - 1s 2ms/step - loss: 0.3690 - accuracy: 0.8717
Epoch 10/10
519/519 [==============================] - 1s 2ms/step - loss: 0.3671 - accuracy: 0.8750

In [ ]:
X_train

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,...,SCC_no,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
9958,-1.202863,0.798937,0.345438,1.039171,0.338364,-0.048349,2.407607,0.636513,0,1,...,1,0,0,0,1,1,0,0,0,0
7841,-0.206438,0.608684,-1.251408,-0.836279,1.756085,-0.048349,1.214691,0.636513,0,1,...,1,0,0,0,1,0,0,0,1,0
9293,-0.352234,1.370087,1.294114,1.039171,0.169396,-0.631241,-0.288525,-1.024344,0,1,...,1,0,0,1,0,0,0,0,1,0
15209,3.016595,-1.377221,-0.299019,-0.836279,0.338364,-1.691863,-1.171141,-1.024344,1,0,...,1,0,0,1,0,1,0,0,0,0
16515,-0.147998,1.142541,0.269619,1.039171,0.338364,1.595165,1.214691,0.636513,0,1,...,1,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,-1.027052,0.254451,-0.282234,0.342987,-0.281329,-0.303677,-0.827543,0.175253,1,0,...,1,0,0,0,1,0,0,0,1,0
11964,-0.323809,0.569868,0.269619,1.039171,0.338364,-0.048349,2.407607,-1.024344,0,1,...,1,0,0,1,0,0,0,0,1,0
5390,-0.499620,-0.919083,-0.753930,-0.836279,0.338364,1.595165,1.214691,-1.024344,0,1,...,1,0,0,1,0,0,0,0,1,0
860,0.379434,-0.574047,0.911789,1.039171,0.338364,1.218360,-1.171141,-0.630211,1,0,...,1,0,0,1,0,0,0,0,1,0


In [ ]:
y_train

9958     2
7841     0
9293     3
15209    2
16515    6
        ..
11284    5
11964    6
5390     5
860      4
15795    2
Name: NObeyesdad, Length: 16606, dtype: int64

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                496       
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 7)                 231       
                                                                 
Total params: 1271 (4.96 KB)
Trainable params: 1271 (4.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train, epochs= 5,validation_split=0.25, verbose=1)

Epoch 1/5
390/390 [==============================] - 1s 3ms/step - loss: 0.3759 - accuracy: 0.8718 - val_loss: 0.3327 - val_accuracy: 0.8810
Epoch 2/5
390/390 [==============================] - 1s 2ms/step - loss: 0.3745 - accuracy: 0.8705 - val_loss: 0.3280 - val_accuracy: 0.8851
Epoch 3/5
390/390 [==============================] - 1s 3ms/step - loss: 0.3714 - accuracy: 0.8710 - val_loss: 0.3280 - val_accuracy: 0.8832
Epoch 4/5
390/390 [==============================] - 1s 3ms/step - loss: 0.3687 - accuracy: 0.8718 - val_loss: 0.3318 - val_accuracy: 0.8829
Epoch 5/5
390/390 [==============================] - 1s 3ms/step - loss: 0.3681 - accuracy: 0.8729 - val_loss: 0.3281 - val_accuracy: 0.8849


In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                496       
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 7)                 231       
                                                                 
Total params: 1271 (4.96 KB)
Trainable params: 1271 (4.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
score = model.evaluate(X_val, y_val, verbose = 1)

130/130 [==============================] - 0s 1ms/step - loss: 0.3612 - accuracy: 0.8772


In [ ]:
prediction = model.predict(test_data)

433/433 [==============================] - 1s 1ms/step


In [ ]:
prediction[33]

array([3.3434139e-10, 5.0264017e-08, 5.1326692e-01, 4.4310412e-01,
       9.1773636e-06, 1.1706955e-03, 4.2449038e-02], dtype=float32)

In [ ]:
len(prediction)

13840

In [ ]:
data = pd.DataFrame(prediction)

In [ ]:
data.round(2)

,0,1,2,3,4,5,6
0,0.00,0.00,0.01,0.99,0.00,0.00,0.00
1,0.00,0.02,0.02,0.00,0.00,0.92,0.04
2,0.00,0.00,0.00,0.00,1.00,0.00,0.00
3,0.00,0.00,0.99,0.01,0.00,0.00,0.00
4,0.00,0.00,0.02,0.00,0.98,0.00,0.00
...,...,...,...,...,...,...,...
13835,0.00,0.04,0.01,0.00,0.00,0.22,0.73
13836,0.00,0.54,0.00,0.00,0.00,0.45,0.01
13837,0.94,0.06,0.00,0.00,0.00,0.00,0.00
13838,0.29,0.70,0.00,0.00,0.00,0.00,0.00


In [ ]:
#data = data.round(2)

In [ ]:
data[0:40]

,0,1,2,3,4,5,6
0,0.00,0.00,0.01,0.99,0.00,0.00,0.00
1,0.00,0.02,0.02,0.00,0.00,0.92,0.04
2,0.00,0.00,0.00,0.00,1.00,0.00,0.00
3,0.00,0.00,0.99,0.01,0.00,0.00,0.00
4,0.00,0.00,0.02,0.00,0.98,0.00,0.00
5,0.98,0.02,0.00,0.00,0.00,0.00,0.00
6,0.80,0.20,0.00,0.00,0.00,0.00,0.00
7,0.00,0.75,0.01,0.00,0.00,0.22,0.02
8,0.00,0.01,0.11,0.00,0.00,0.10,0.78
9,0.04,0.87,0.00,0.00,0.00,0.09,0.00


In [ ]:
data =data.idxmax(axis=1)

In [ ]:
data[0]

3

In [ ]:
list1 = data.apply(lambda row: row[row.max()].index.tolist(), axis=1)

KeyError: 0.99

In [ ]:
y_predection = le.inverse_transform(data)

In [ ]:
y_predection

array(['Obesity_Type_II', 'Overweight_Level_I', 'Obesity_Type_III', ...,
       'Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_II'],
      dtype=object)

In [ ]:
submission = pd.DataFrame({
    'id': test_ids,
    'Exited': pd.Series(y_predection)
})

In [ ]:
submission

,id,Exited
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Normal_Weight
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [ ]:
submission[submission['Exited'].isna()]

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index=False)